In [ ]:
'''Open in google colab please'''

%matplotlib notebook
import pandas as pd
import csv
import base64
from zipfile import ZipFile
from IPython.display import display, clear_output
import ipywidgets as widgets


class ExcelToCSV_Dialog():

  def __init__(self):
    self.excel_file = None
    self.file_hboxes = []
    self.start_dialog()
    self.filename = ""


  def on_clear(self, _):
    self.file_hboxes = []
    self.excel_file = None
    self.uploader.value.clear()
    self.uploader._counter = 0

  def readout_checkbox_values(self):
    """
      returns a list of sheetnames, which are to be parsed from the
      specified excel_file
    """
    return [cb.description for cb in self.checkboxes if cb.value]

  def return_function(self, *args):
    pass


  def select_all_function(self, *args):
    for cb in self.checkboxes:
      cb.value=True

  def display_sheet_checkboxes(self):

    for name in self.excel_file.sheet_names:

      try:
        df = self.excel_file.parse(sheet=name)
        res = df.to_csv(index=False)

        filename = name + ".csv"
        b64 = base64.b64encode(res.encode())
        payload = b64.decode()
        html_button = '''<html>
        <head>
        <meta name="viewport" content="width=device-width, initial-scale=1">
        </head>
        <body>
        <a download="{filename}" href="data:text/csv;base64,{payload}" download>
        <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Download File</button>
        </a>
        </body>
        </html>
        '''
        button = widgets.HTML(html_button.format(payload=payload,filename=filename))
        self.file_hboxes.append(widgets.HBox([widgets.Text(name), button]))

      except:
        # sometimes, excel sheets can't be turned into csv files,e.g. when
        # the excel sheet contains a diagram
        continue

    display(widgets.VBox(self.file_hboxes))





  def upload_excel_file(self, *args):

    with self.out:

      self.out.clear_output()
      if not self.uploader.value.keys():
        print('pls upload a file first')
        return

      tmp_file = self.uploader.value[list(self.uploader.value.keys())[0]]
      self.filename = tmp_file["metadata"]["name"]
      content = tmp_file["content"]
      self.excel_file = pd.ExcelFile(content)
      self.display_sheet_checkboxes()


  def start_dialog(self):
    self.uploader = widgets.FileUpload(
      accept='.xls',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
      multiple=False,  # True to accept multiple files upload else False
      description='select xls file'
    )

    self.clear = widgets.Button(description='Clear!', layout=widgets.Layout(width='100px'));
    self.clear.on_click(self.on_clear)
    self.file = widgets.HBox([widgets.Label(value="File:"), self.uploader, self.clear])

    self.upload_excel_button = widgets.Button(description='upload', layout=widgets.Layout(width='200px'));
    self.upload_excel_button.on_click(self.upload_excel_file)
    self.out = widgets.Output()

    display(self.file, self.upload_excel_button, self.out)




#@title XLS to CSV { vertical-output: true }
a = ExcelToCSV_Dialog()

HBox(children=(Label(value='File:'), FileUpload(value={}, accept='.xls', description='select xls file'), Butto…

Button(description='upload', layout=Layout(width='200px'), style=ButtonStyle())

Output()


